# Fine tuned model inference

In [ ]:
# Packages
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from peft import PeftModel
import torch

In [ ]:
# Load model
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
adapter_path = "Tinyllama-ft-qlora-dsz3/checkpoint-208"  # Replace with your adapter's path or identifier

# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# Load the base model with quantization
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

# Load the adapter into the base model
model = PeftModel.from_pretrained(base_model, adapter_path)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Define the model to infer
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")


In [ ]:
# Inference

# We use the tokenizer's chat template to format
messages = [
    {"role": "system","content": "CREATE TABLE head(age INTEGER)"},
    {"role": "user", "content": "How many heads of the departments are older than 56?"},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

In [ ]:
# Merge adapter weights into the base model
merged_model = model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("./merged_model")